# Finding Cycles in NFT Transactions

## Overview

A Non-fungible Token (NFT) is a digital asset which cannot be copied, substituted, or subdivided, meaning it is wholly unique.

![title](http://web.archive.org/web/20220328080727im_/https://blog.portion.io/content/images/2021/07/Kevin-McCoy-Quantum.gif)

NFT’s are stored on blockchain systems, notably Ethereum, with the owner of an NFT being whomever has access to the wallet it is stored within.

Since the first NFT, Quantum (shown above), many new NFT artworks have arisen including Cryptokitties and Cryptopunks. This culminated in a boom in 2021 which included the arrival of the Bored Ape Yacht Club, with NFTS reaching a global market valued of $15.7 Billion.

In this example we will analyse an NFT dataset in order to find suspicious trading cycles.

1. Using Raphtory to load the NFT data as a graph
2. Execute a cycle detection algorithm
3. Analyse the results to find suspicious cycles of NFTs

The data we used is a cleaned and trimmed version from [1]


[1] Nadini, M., Alessandretti, L., Di Giacinto, F. et al. Mapping the NFT revolution: market trends, trade networks, and visual features. Sci Rep 11, 20902 (2021). https://doi.org/10.1038/s41598-021-00053-8

## Loading data with Raphtory

First we install some extra packages

In [1]:
pip install scipy seaborn

Note: you may need to restart the kernel to use updated packages.


Now we import the following so we can use Raphtory

In [2]:
import time
from calendar import timegm
from pyraphtory.context import PyRaphtory
from pyraphtory.builder import *
from pyraphtory.spouts import FileSpout
from pyraphtory.sinks import *
from pyraphtory.formats import JsonFormat
import csv
from nft_helper import *

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment AdoptOpenJDK-11.0.11+9 (build 11.0.11+9)
OpenJDK 64-Bit Server VM AdoptOpenJDK-11.0.11+9 (build 11.0.11+9, mixed mode)


Java found!
Getting JAVA_HOME
JAVA_HOME found = /Users/haaroony/.sdkman/candidates/java/current/bin/java
14:10:55.725 [io-compute-blocker-8] INFO  s.c.c.c.Py4JServer - Starting PythonGatewayServer...


Below are our data sources
- `date_price` a dictionary containing the daily ETH-USD price, we we need to convert as some prices are missing
- `filename` - this is the cleaned transaction data, which represents NFTs being bought by wallets 
- `at_time` - is the time we want to run our analysis, this is the last time in the dataset 

#### Run this below if you want the full data, turn this into a code block

!curl -o Data_API_reduced.csv https://osf.io/download/kaumt/ -L

#### Run this below if you want the full data, turn this into a code block

!curl -o Data_API_reduced.csv https://osf.io/download/kaumt/ -L

In [4]:
eth_price_csv = 'ETH-USD.csv'
filename = "Data_API_reduced.csv"
at_time = 1561661534

date_price = setup_date_prices(eth_price_csv)

For this example, we are going to create a bipartite graph. 

In our graph, we have wallets on the left, and NFTs on the right. 

Each edge represents that an NFT was purchased by a wallet. 

The `parse_graph` function will take each line, from the spout, and turn these into graph updates. 


![title](https://www.raphtory.com/images/nfts/buy_nft.png)


In [5]:
def parse_graph(graph, line):
    file_line = line.split(',')
    if file_line[0] == "Smart_contract":
        return
    # Seller details
    seller_address = file_line[3]
    seller_address_hash = graph.assign_id(seller_address)
    # Buyer details
    buyer_address = file_line[5]
    buyer_address_hash = graph.assign_id(buyer_address)
    # Transaction details
    datetime_str = file_line[13]
    timestamp_utc = time.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    timestamp = timegm(timestamp_utc)
    tx_hash = file_line[2]
    token_id_str = file_line[1]
    token_id_hash = graph.assign_id(token_id_str)

    crypto = file_line[8]
    if crypto != 'ETH':
        return

    if file_line[9] == "":
        price_usd = date_price[datetime_str[0:10]]
    else:
        price_usd = float(file_line[9])

    # NFT Details
    collection_cleaned = file_line[14]
    market = file_line[11]
    category = file_line[15]

    #  add buyer node
    graph.add_vertex(
        timestamp,
        buyer_address_hash,
        Properties(ImmutableProperty("address", buyer_address)),
        Type("Wallet")
    )

    # Add node for NFT
    graph.add_vertex(
        timestamp,
        token_id_hash,
        Properties(
            ImmutableProperty("id", token_id_str),
            ImmutableProperty("collection", collection_cleaned),
            ImmutableProperty("category", category)
        ),
        Type("NFT")
    )

    # Creating a bipartite graph,
    # add edge between buyer and nft
    graph.add_edge(
        timestamp,
        buyer_address_hash,
        token_id_hash,
        Properties(
            StringProperty("transaction_hash", tx_hash),
            StringProperty("crypto", crypto),
            StringProperty("price_usd", str(price_usd)),
            StringProperty("market", market),
            StringProperty("token_id", token_id_str),
            StringProperty("buyer_address", buyer_address)
        ),
        Type("Purchase")
    )

Now we will start a raphtory instance and load our data

In [6]:
graph = PyRaphtory.local().new_graph()

graph.load(Source(FileSpout(filename), GraphBuilder(parse_graph)))

14:11:05.130 [io-compute-blocker-1] INFO  s.c.c.c.c.IngestionServiceImpl$ - Starting Ingestion Service
14:11:05.160 [io-compute-blocker-1] INFO  s.c.c.c.c.PartitionServiceImpl$ - Starting partition service for id '0'
14:11:05.170 [io-compute-blocker-8] INFO  s.c.c.c.c.QueryOrchestrator - Starting Query Service
14:11:05.239 [io-compute-blocker-9] INFO  s.c.c.c.Prometheus$ - Prometheus started on port /0:0:0:0:0:0:0:0:1737
14:11:05.438 [spawner-akka.actor.default-dispatcher-5] INFO  s.c.c.c.c.QueryOrchestrator - Deploying new Query Manager for graph 'useful_lavender_quelea' - request by 'cheerful_plum_stork' 
14:11:05.793 [io-compute-0] INFO  s.c.c.FileSpoutInstance - Spout: Processing file 'Data_API_reduced.csv' ...
14:11:05.807 [spawner-akka.actor.default-dispatcher-10] INFO  s.c.c.c.c.QueryManager - Source '1' is blocking analysis for Graph 'useful_lavender_quelea'


com.raphtory.api.analysis.graphview.PyDeployedTemporalGraph@687aacc5

## Executing a Cycle detection algorithm

Below is a visual representation of the cycle detection. 

This is when a NFT has been sold by a wallet, but eventually ends up being owned by it again. This can involve a chain of any number of other wallets, take any amount of time, and increase or decrease the price by any amount.

Below we can see an example of such a cycle where Wallet A sold an NFT to Wallet B for $50$ on Monday, but its back in their position by Thursday, costing them $750$. They either have serious sellers remorse or, more likely, the same person owns all these wallets and is trying to pump up the price of the given NFT.

Our algorithm:
- Gets each NFT
- For each time it was purchased, checks the list of purchases to see if a user re-bought the NFT at a later date, at a higher price

This all happens within the `step` function. A `step` is executed once on each node.  

The code has been optimised to O(n) complexity. 

![title](https://www.raphtory.com/images/nfts/NFT_Cycle.png)

In [7]:
from pyraphtory.algorithm import PyAlgorithm
from pyraphtory.graph import TemporalGraph, Row, Table
from pyraphtory.vertex import Vertex
import pyraphtory.scala.collection

CYCLES_FOUND: str = "CYCLES_FOUND"


class CycleMania(PyAlgorithm):
    def __call__(self, graph: TemporalGraph) -> TemporalGraph:
        def step(v: Vertex):
            if v.type() != "NFT":
                v[CYCLES_FOUND] = []
                return
            all_cycles = []
            all_purchases = sorted(v.explode_in_edges(), key=lambda e: e.timestamp())
            purchasers = list(map(lambda e:
                                  dict(price_usd=float(e.get_property_or_else("price_usd", 0.0)),
                                       nft_id=e.get_property_or_else("token_id", "_UNKNOWN_"),
                                       tx_hash=e.get_property_or_else("transaction_hash", ""),
                                       time=e.timestamp(),
                                       buyer=e.get_property_or_else("buyer_address", "_UNKNOWN_")),
                                  all_purchases))
            if len(purchasers) > 2:
                buyers_seen = {}
                for pos, item_sale in enumerate(purchasers):
                    buyer_id = item_sale['buyer']
                    if buyer_id not in buyers_seen:
                        buyers_seen[buyer_id] = pos
                    else:
                        prev_pos = buyers_seen[buyer_id]
                        prev_price = purchasers[prev_pos]['price_usd']
                        current_price = item_sale['price_usd']
                        buyers_seen[buyer_id] = pos
                        if prev_price < current_price:
                            all_cycles.append(purchasers[prev_pos:pos + 1])
            if len(all_cycles):
                v[CYCLES_FOUND] = all_cycles
            else:
                v[CYCLES_FOUND] = []

        return graph.reduced_view().step(step)

    def tabularise(self, graph: TemporalGraph):
        def get_cycles(v: Vertex):
            vertex_type = v.type()
            rows_found = [Row()]
            if vertex_type == "NFT" and len(v[CYCLES_FOUND]):
                nft_id = v.get_property_or_else('id', '_UNKNOWN_')
                cycles_found = v[CYCLES_FOUND]
                nft_collection = v.get_property_or_else('collection', '_UNKNOWN_')
                nft_category = v.get_property_or_else('category', '_UNKNOWN_')
                rows_found = list(map(lambda single_cycle:
                                      Row(
                                          nft_id,
                                          nft_collection,
                                          nft_category,
                                          len(single_cycle),
                                          dict(cycle={'sales': single_cycle},
                                               profit_usd=float(single_cycle[len(single_cycle) - 1]['price_usd']) -
                                                          float(single_cycle[0]['price_usd']),
                                               buyer=str(single_cycle[0]['buyer']))
                                      ), cycles_found))
            return rows_found

        return graph.explode_select(lambda v: get_cycles(v)).filter(lambda row: len(row.get_values()) > 0)


We run the algorithm as follows 

In [ ]:
graph \
    .at(at_time) \
    .past() \
    .execute(CycleMania()) \
    .write_to(FileSink('/tmp/raphtory_nft_python', format=JsonFormat())) \
    .wait_for_job()

14:11:06.647 [io-compute-3] INFO  s.c.c.c.c.QueryExecutor - CycleMania_124528886998707601_0: Starting QueryExecutor.
14:11:06.682 [spawner-akka.actor.default-dispatcher-11] INFO  s.c.c.c.c.QueryManager - Query 'CycleMania_124528886998707601' currently blocked, waiting for ingestion to complete.
14:11:09.007 [spawner-akka.actor.default-dispatcher-11] INFO  s.c.c.c.c.QueryManager - Source '1' is unblocking analysis for Graph 'useful_lavender_quelea' with 30 messages sent. Latest update time was 1541029113
14:11:09.092 [spawner-akka.actor.default-dispatcher-6] INFO  s.c.c.c.c.QueryManager - Source 1 has completed ingesting and will now unblock
14:11:09.093 [spawner-akka.actor.default-dispatcher-6] INFO  s.c.c.c.c.QueryManager - Query 'CycleMania_124528886998707601' received, your job ID is 'CycleMania_124528886998707601'.


## Analysing the results

First we import the tools we are using for analysis

In [ ]:
import pandas as pd
import json
import seaborn as sns
from scipy import stats
import numpy as np
import time
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '16'
from nft_helper import *
from collections import Counter

### Read data

We will now read the data that Raphtory produced. 

Please check which path the data was saved too, and adjust the line below. 

In [ ]:
import glob
for filename in glob.glob('/tmp/raphtory_nft_python/*/partition-0.json'):
    with open(filename,'r') as f:
        data = load_json(filename)

new_data = []
# filter any cycles that are less than 2 hops
for d in data:
    if d['row'][3] > 2:
        new_data.append(d)
data = new_data
data_df = pd.DataFrame(data)

In [ ]:
data_df["profit"]=data_df["row"].apply(lambda x: x[4]['profit_usd'])
data_df["min_ts"] = data_df["row"].apply(lambda x: min(map(lambda y: y["time"],x[4]['cycle']['sales'])))
data_df["max_ts"] = data_df["row"].apply(lambda x: max(map(lambda y: y["time"],x[4]['cycle']['sales'])))
data_df["length"] = data_df["row"].apply(lambda x: len(x[4]['cycle']['sales']))
data_df["duration_days"] = (data_df["max_ts"] - data_df["min_ts"])/86400
display(data_df)

## Cycle length distributions

Below is a simple CDF showing the distribution of the lengths of cycles. 

In [ ]:
x,y = ccdf(data_df.duration_days)

fig, ax = plt.subplots()
ax.set_xscale("log")
ax.set_yscale("log")
ax.plot(x,y, marker="o")
ax.set_ylabel("CCDF")
ax.set_xlabel("Cycle duration (days)")

plt.show()

In [ ]:
x,y = ccdf(data_df.length)

fig, ax = plt.subplots()
ax.set_xscale("log")
ax.set_yscale("log")
ax.plot(x,y, marker="o")
ax.set_ylabel("CCDF")
ax.set_xlabel("Cycle length (hops)")

plt.show()

## How many traders took part in cycles?

Above, we can see that 20 traders participated in over 528 cycles each. 

Lets write some code to inspect these NFT cycles in more depth. 

The `pretty_cycle` function takes a cycle and prints out each subsequent trade and the profit and time between trades.  

In [ ]:
def pretty_cycle(cycle):
    as_string = '   '
    prev_price = cycle[0]['price_usd']
    prev_time = cycle[0]['time']
    for item in cycle:
        diff = item['price_usd']-prev_price
        time_secs = item['time']-prev_time
        time_mins = time_secs/60
        time_hours = time_secs/60/60
        time_days = time_secs/60/60/24 
        prev_time = item['time']
        time_str = '%.1fm/%.1fh/%.2fd' % (time_mins, time_hours, time_days)
        as_string += 'T(d) '+time_str+', B: '+item['buyer'][:4]+'.. $'+str(item['price_usd'])+'('+str(diff)+') '+item['tx_hash']+'\n->'
    as_string = as_string[:-3]
    print(as_string)

# Lets take a look at the trader who did the most cycles

In [ ]:
largest_trader_addr = traders_count_sorted[0] # '0x8acc1421ec98689461ff5777de8ad6648dc6d643'
largest = []
for trade in data:
    # ignore any rari tokens, as they are fractionalised
    #. ignore any short trades 
    if '_Rari' in trade['row'][0] or trade['row'][3] <= 2:
        continue
    if trade['row'][4]['buyer'] == largest_trader_addr:
        largest.append(trade)

#### Which NFTs did they trade?

In [ ]:
from collections import Counter
most_common_nft_collection = []
most_common_nft = []
for trade in largest:
    nft_collection =  trade['row'][0].split('_')
    if len(nft_collection) > 2:
        print("crap")
    most_common_nft.append(trade['row'][0])
    most_common_nft_collection.append(nft_collection[1])
most_common_nft_collection = set(most_common_nft_collection)
most_common_nft_collection = Counter(most_common_nft_collection)
most_common_nft = Counter(most_common_nft)
most_common_nft_collection

#### What happens when we look at their cycles in more detail?

In [ ]:
pretty_cycle(largest[10]['row'][4]['cycle']['sales'])

We have found something here, this trader trades each nft between wallets within a ~4 hour window??

#### What was the time between each transaction?

In [ ]:
import math
def time_finder(cycle):
    prev_time = cycle[0]['time']
    times = []
    for sale in cycle:
        time_secs = sale['time']-prev_time
        time_hours = time_secs/60/60
        prev_time = sale['time']
        time_hours = round(time_hours, 1)
        times.append(time_hours)
    return times

time_found = []
for trade in largest:
    time_found.extend(time_finder(trade['row'][4]['cycle']['sales']))
time_found = Counter(time_found)
print("total times : %i" % sum(time_found.values()))
between_39_41 = time_found[3.4]+time_found[3.5]+time_found[3.6] +time_found[3.7] +time_found[3.8] +time_found[3.9] +time_found[4.0] +time_found[4.1] +time_found[4.2]+time_found[4.3]+time_found[4.4]+time_found[4.5]+time_found[4.6]
print("times between 3.4 and 4.6 : %i - %.2fp" % (between_39_41, between_39_41/sum(time_found.values())*100))

#### Who did they trade with?

In [ ]:
wallet_interactions = []
for trade in largest:
    for sale in trade['row'][4]['cycle']['sales']:
        wallet_interactions.append(sale['buyer'])
wallet_interactions = Counter(wallet_interactions)
sorted(wallet_interactions.items(), key=lambda x: x[1], reverse=True), len(wallet_interactions)

Interestingly they only traded their NFTs with 20 other wallets. 

All of these traders also participated in many cycles. 

#### What were the patterns of the fellow traders cycles?

In [ ]:
import random

fellow_traders
print(len(fellow_traders))

others_sibs = {}
for sibling in fellow_traders:
    others_sibs[sibling] = []
    for trade in data:
        if '_Rari' in trade['row'][0] or trade['row'][3] <= 2:
            continue
        if trade['row'][4]['buyer'] == sibling:
            others_sibs[sibling].append(trade)
    print(sibling)
    rand = random.choice(range(0,len(others_sibs[sibling])))
    pretty_cycle(others_sibs[sibling][rand]['row'][4]['cycle']['sales'])
    print()

All of these traders also follow the same, or a similar pattern.

Trading with each other, waiting ~4 hours before trading again. 